In [2]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import random
import tensorflow as tf
#from bayes_opt import BayesianOptimization
from skopt import gp_minimize
from skopt.space import Real, Integer
from functools import partial
import requests #for the API

2025-03-12 15:03:48.647410: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
overall_dict = {}

overall_dict["Total Fat"] = 0
overall_dict["Saturated Fat"] = 0
overall_dict["Trans Fat"] = 0
overall_dict["Cholesterol"] = 0
overall_dict["Sodium"] = 0
overall_dict["Total Carbs"] = 0
overall_dict["Fiber"] = 0
overall_dict["Total Sugar"] = 0
overall_dict["Protein"] = 0
overall_dict["Calcium"] = 0
overall_dict["Iron"] = 0
overall_dict["Potassium"] = 0
overall_dict["Vitamin D"] = 0
overall_dict["Water"] = 0
overall_dict["Weight"] = 100
overall_dict["Energy/Calories"] = 0

key_list_API = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Sodium', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Water', 'Weight', 'Energy/Calories']

In [4]:
tates_dict = overall_dict.copy()
tates_dict["Total Fat"] = 7
tates_dict["Saturated Fat"] = 4.5
tates_dict["Trans Fat"] = 0
tates_dict["Cholesterol"] = .025
tates_dict["Sodium"] = .16
tates_dict["Total Carbs"] = 18
tates_dict["Fiber"] = .8
tates_dict["Total Sugar"] = 12
tates_dict["Protein"] = 2
tates_dict["Calcium"] = .0000001
tates_dict["Iron"] = .0009
tates_dict["Potassium"] = .01
tates_dict["Vitamin D"] = .05
tates_dict["Weight"] = 28
tates_dict["Energy/Calories"] = 140

In [5]:
natural_vanilla_dict = overall_dict.copy()
#calories per 10 grams: 27 

natural_vanilla_dict["Sodium"] = .04 / 100
natural_vanilla_dict["Total Carbs"] = 12
natural_vanilla_dict["Weight"] = 100
natural_vanilla_dict["Energy/Calories"] = 270

baking_soda_alex_dict = overall_dict.copy()
#calories per 10 grams: 0

baking_soda_alex_dict["Sodium"] = 27.36
baking_soda_alex_dict["Weight"] = 100

In [6]:
def unit_conversion(nutrient_name,dict,amount,unit):
    if unit == "kJ":
        dict[nutrient_name] = (amount/4.184)
    elif unit == "mg":
        dict[nutrient_name] = (amount/1000)
    elif unit == "µg":
        dict[nutrient_name] = (amount/ 1000000)
    else:
        dict[nutrient_name] = (amount)
    return None

In [7]:
total_fats = ["Total lipid (fat)"]
saturated_fats = ["Saturated fatty acids","Fatty acids, total saturated"]
trans_fat = ["Fatty acids, total trans"]
cholesterol = ["Cholesterol"]
sodium = ["Sodium, Na"]
total_carbs = ["Carbohydrate, by difference"]
fiber = ["Fiber","Fiber, total dietary"]
total_sugar = ["Sugars, Total","Total Sugars","Total Sugar","Sugars, added"]
protein = ["Protein"]
calcium = ["Calcium, Ca"]
iron = ["Iron, Fe"]
potassium = ["Potassium, K"]
vitamin_D = ["Vitamin D (D2 + D3)"]
weight = ["Weight"]
energy_calories = ["Energy"] 

def super_big_function(dict,amount,nutrient_name,unit):
    if nutrient_name in total_fats:
        unit_conversion("Total Fat",dict,amount,unit)
    elif nutrient_name in saturated_fats:
        unit_conversion("Saturated Fat",dict,amount,unit)
    elif nutrient_name in trans_fat:
        unit_conversion("Trans Fat",dict,amount,unit)
    elif nutrient_name in cholesterol:
        unit_conversion("Cholesterol",dict,amount,unit)   
    elif nutrient_name in sodium:
        unit_conversion("Sodium",dict,amount,unit)
    elif nutrient_name in total_carbs:
        unit_conversion("Total Carbs",dict,amount,unit)
    elif nutrient_name in fiber:
        unit_conversion("Fiber",dict,amount,unit)
    elif nutrient_name in total_sugar:
        unit_conversion("Total Sugar",dict,amount,unit)  
    elif nutrient_name in protein:
        unit_conversion("Protein",dict,amount,unit)
    elif nutrient_name in calcium:
        unit_conversion("Calcium",dict,amount,unit)
    elif nutrient_name in iron:
        unit_conversion("Iron",dict,amount,unit)
    elif nutrient_name in potassium:
        unit_conversion("Potassium",dict,amount,unit)   
    elif nutrient_name in vitamin_D:
        unit_conversion("Vitamin D",dict,amount,unit)
    elif nutrient_name in weight:
        unit_conversion("Weight",dict,amount,unit)
    elif nutrient_name in energy_calories:
        unit_conversion("Energy/Calories",dict,amount,unit)
    return None

In [8]:
import re

In [9]:
new_key_list = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Sodium', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Water', 'Weight']

In [10]:
api_key = "pJdDYwC0EdXMpR18wMXFqjMxO4zYGCuTOfNfCSH4"

def getting_from_API(query):
    food_words = query.split()
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={query}&dataType=Foundation&api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    for individual_food_dict in data['foods'][:5]:
        match = 0
        individual_food = individual_food_dict["description"].lower()
        individual_words_list = re.split(r"[,\s]+", individual_food)
        #individual_words_list = individual_food.split()
        
        for word in food_words:
            if word not in individual_words_list:
                match = match - 1
            else:
                match = match + 1
        if match == len(food_words):
            return individual_food_dict['fdcId']
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={query}&dataType=Branded&api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    for individual_food_dict in data['foods'][:5]:
        match = 0
        individual_food = individual_food_dict["description"].lower()
        individual_words_list = re.split(r"[,\s]+", individual_food)
        for word in food_words:
            if word not in individual_words_list:
                match = match - 1
            else:
                match = match + 1
        if match == len(food_words):
            return individual_food_dict['fdcId']       
    
    return "ERROR NOTHING IS A MATCH :((((("

def opening_API(fdc_id,dict):
    
    nutrition_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}?api_key={api_key}"

    response = requests.get(nutrition_url)
    nutrition_data = response.json()

    for nutrient in nutrition_data['foodNutrients']:
        nutrient_name = nutrient['nutrient']['name']
        amount = nutrient.get("amount","NA")
        unit = nutrient.get("nutrient", {}).get("unitName", "")
        super_big_function(dict,amount,nutrient_name,unit)
    return dict

In [11]:
getting_from_API("semi-sweet chocolate")

2012567

In [12]:
queries = ["semi-sweet chocolate","unbleached flour","salted butter","cane sugar","brown cane sugar","egg whole","salt"]

def product_dicts(queries):
    dicts = {}
    for query in queries:
        fdc_id = getting_from_API(query)
        print(fdc_id)
        new_dict = overall_dict.copy()
        opening_API(fdc_id,new_dict)
        dicts[query] = new_dict
        print("added")
    return dicts


#semi_sweet_chocolate_dict = overall_dict.copy()
#fdc_id_semi_sweet_chocolate = getting_from_API("semi sweet chocolate")
#opening_API(fdc_id_semi_sweet_chocolate,semi_sweet_chocolate_dict)

#dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict,baking_soda_alex_dict,salt_dict,natural_vanilla_dict]
#heavy_dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict]
#key_list = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Sodium","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Weight","Calories"]
#key_list_wo_weight = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Sodium","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Calories"]
#key_list_wo_sodium_and_weight = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Calories"]

In [13]:
tates_ingredient_dict = product_dicts(queries)

2012567
added
1104913
added
790508
added
1882741
added
2015023
added
748967
added
746775
added


In [14]:
tates_ingredient_dict["baking soda"] = baking_soda_alex_dict
tates_ingredient_dict["vanilla extract"] = natural_vanilla_dict
tates_ingredient_dict["salted butter"]["Energy/Calories"] = 714
ingredients = queries[:-1]
print(queries)
print(ingredients)
ingredients.append("baking soda")
ingredients.append("salt")
ingredients.append("vanilla extract")
print(ingredients)

['semi-sweet chocolate', 'unbleached flour', 'salted butter', 'cane sugar', 'brown cane sugar', 'egg whole', 'salt']
['semi-sweet chocolate', 'unbleached flour', 'salted butter', 'cane sugar', 'brown cane sugar', 'egg whole']
['semi-sweet chocolate', 'unbleached flour', 'salted butter', 'cane sugar', 'brown cane sugar', 'egg whole', 'baking soda', 'salt', 'vanilla extract']


In [15]:
heavy_ingredients = ingredients[:-3]
print(heavy_ingredients)

['semi-sweet chocolate', 'unbleached flour', 'salted butter', 'cane sugar', 'brown cane sugar', 'egg whole']


In [16]:
def building_x_list(number,dictionary_list):
    x_list = []
    for variable in range(0, len(dictionary_list)):
        x_list.append(number)
    return x_list

x_list = building_x_list(1,ingredients)


In [17]:
def equations(product_dictionaries,ingredients,x_list,desired_dict,key_list):
    list_of_equations = []
    for key in key_list:
        equation = 0
        for i,ingredient in enumerate(ingredients):
            equation = equation + (product_dictionaries[ingredient][key] * (x_list[i] / 100))
        list_of_equations.append(equation-(desired_dict[key]))
        #so if something iss negative, tates > val, if positive, val > tates. so we only want negative!
    return list_of_equations

In [28]:
equations(tates_ingredient_dict,heavy_ingredients,trial,tates_dict,heavy_key_list_no_weight)

[0.6160736322879785,
 -0.36973636894226125,
 0.0,
 -0.0033023502933979057,
 0.07625149765014427,
 -0.32515838432312016,
 0.4589569746494284,
 -0.9965880723953249,
 0.0032110334133148197,
 -0.0004691434089303016,
 0.0027104968500137328,
 -0.049999933759455134,
 0.0,
 -0.06572284098336922]

In [19]:
def initialize_variables_tf(n,desired_dict,first_ratio_range):
    #first_ratio_range is a tuple, low and high ends of 
    largest_weight = desired_dict["Weight"]
    first_variable_weight = 0
    other_weight_for_first_ratio = largest_weight
    lower_bound, upper_bound = first_ratio_range
    #this is for ex. cho chips are 80 percent of dough instead of cho chips are 80 percent of the whole weight
    first_ratio = first_variable_weight / other_weight_for_first_ratio

    while (first_ratio < lower_bound) or (first_ratio > upper_bound):
        initial_values = tf.random.uniform(shape=(n,), minval=0.0, maxval=largest_weight)
        initial_values = tf.sort(initial_values, direction='DESCENDING')

        initial_values = (initial_values * largest_weight) / tf.reduce_sum(initial_values)
        first_variable_weight = initial_values[0]
        other_weight_for_first_ratio = tf.reduce_sum(initial_values[1:])
        first_ratio = first_variable_weight / other_weight_for_first_ratio

    return tf.Variable(initial_values, trainable=True)

In [20]:
def loss_tf(variables_tf,predicted):

    error = tf.reduce_mean(tf.square(predicted))
    error = error +  5*(tf.reduce_sum(tf.square(tf.minimum(variables_tf, 0))))
    
    variables_tf_array = variables_tf.numpy()
    for i in range(0,len(variables_tf_array)-2):
        if variables_tf_array[i] - variables_tf_array[i + 1] < 0:
            error = error + ((variables_tf_array[i] - variables_tf_array[i + 1]) ** 2)

    return error

In [21]:
#how to ensure returns either negative or 0 error? how to penalize for positive?

def running_tf(product_dictionaries,ingredients,desired_dict,key_list,ratio_range,print=False):  
    #number of ingredients
    n = len(ingredients)
    #first guess
    variables_tf = initialize_variables_tf(n,desired_dict,ratio_range)
    #the optimizer chosen: stochastic gradient descent. WHAT IF WE TRIED batch gradient descent here?
    #pros/cons: https://www.geeksforgeeks.org/difference-between-batch-gradient-descent-and-stochastic-gradient-descent/
    #whats the math behind this?
        #updates each variable. the variable is updated to be its value - 
        #the learning rate * the gradient of how much that variable affects the loss.
        #THIS PART IS DIFFERENT FROM MY PREVIOUS METHOD MATHEMATICALLY, but does the same thing
        #SHOW tests for learning rates. date the comments.
    optimizer = tf.keras.optimizers.SGD(learning_rate=.01)

# Training loop
    #redefines the variable 600 times. what number is OPTIMAL?
    for epoch in range(400):

        #the "with" allows for automatic differentiation. this means the with part locks in the operations. but here no gradients are calculated.
        with tf.GradientTape() as tape:
        
            predictions1 = equations(product_dictionaries,ingredients,variables_tf,desired_dict,key_list)
            predictions = tf.convert_to_tensor(predictions1)
            loss = loss_tf(variables_tf,predictions)

        #gradients are computed here.
        gradients = tape.gradient(loss, [variables_tf])

        #apply these gradients, perform sgd on our variables (which are being updated) 
        optimizer.apply_gradients(zip(gradients, [variables_tf]))

        predictions_updated = equations(product_dictionaries, ingredients, variables_tf, desired_dict, key_list)
        new_loss = loss_tf(variables_tf, tf.convert_to_tensor(predictions_updated))

        if print == True and epoch % 20 == 0:
            print((tf.reduce_sum(variables_tf).numpy()))
            print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}, New Loss: {new_loss.numpy():.4f}, Variables: {variables_tf.numpy()}")

        if new_loss > loss:
            return (loss,variables_tf.numpy().tolist())
            
    return (loss,variables_tf.numpy().tolist())

In [22]:
heavy_key_list = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Water', 'Weight', 'Energy/Calories']

In [23]:
heavy_key_list_no_weight = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Water', 'Energy/Calories']

In [24]:
heavy_key_list_no_weight_or_cal = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Water']

In [25]:
key_list_no_weight = ['Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Sodium', 'Total Carbs', 'Fiber', 'Total Sugar', 'Protein', 'Calcium', 'Iron', 'Potassium', 'Vitamin D', 'Energy/Calories']

In [94]:
running_tf(tates_ingredient_dict,heavy_ingredients,tates_dict,new_key_list,(.25,.45))

(<tf.Tensor: shape=(), dtype=float32, numpy=0.1413825>,
 [6.597270965576172,
  6.529214382171631,
  5.8894124031066895,
  5.591473579406738,
  3.426011085510254,
  0.3141237795352936])

In [26]:
def multiple_tfs(product_dictionaries,ingredients,desired_dict,key_list,ratio_range,number_of_times_run):
    lowest_loss = 500
    lowest_resulting_list = []
    for i in range(0,number_of_times_run):
        loss, resulting_list = running_tf(product_dictionaries,ingredients,desired_dict,key_list,ratio_range)
        if loss < lowest_loss:
            lowest_loss = loss
            lowest_resulting_list = resulting_list
    return (lowest_loss,lowest_resulting_list)

In [27]:
trial =  [6.687910079956055,
  6.379572868347168,
  6.321332931518555,
  5.336599349975586,
  4.272467613220215,
  1.6648460626602173]

In [26]:
def add_noise(optimized_heavy_ingredients):
    optimized_w_noise = []
    for i in range(0,len(optimized_heavy_ingredients)):
        number = random.randint(1,2)
        if number == 1:
            optimized_w_noise.append(optimized_heavy_ingredients[i] - .1)
        else:
            optimized_w_noise.append(optimized_heavy_ingredients[i] + .1)
    return optimized_w_noise

In [97]:
optimized_dictionary={}


def fixed_heavy_dictionary(optimized_heavy_ingredients,key_list_initial, key_list_final):

    x_list = building_x_list(1,heavy_ingredients)

    error_for_optimized = equations(tates_ingredient_dict,heavy_ingredients,optimized_heavy_ingredients,tates_dict,key_list_initial)
    print(sum(error_for_optimized))

    #evaluate on initial key list, but return all keys. So this means then evaluating the keys that weren't included in the initial key_list
    
    for i in range(0,len(key_list_final)):
        key = key_list[i]
        print(key)
        error_at_this_key = error_for_optimized[i]
        optimized_dictionary[key] = error_at_this_key

    return optimized_dictionary
        

In [95]:
triall =   [6.597270965576172,
  6.529214382171631,
  5.8894124031066895,
  5.591473579406738,
  3.426011085510254,
  0.3141237795352936]

In [91]:
fixed_heavy_dictionary(trial,key_list_API)

1.8869396080536853


{'Total Fat': 0.6160736322879785,
 'Saturated Fat': -0.36973636894226125,
 'Trans Fat': 0.0,
 'Cholesterol': -0.0033023502933979057,
 'Sodium': -0.12201583836913109,
 'Total Carbs': 0.07625149765014427,
 'Fiber': -0.32515838432312016,
 'Total Sugar': 0.4589569746494284,
 'Protein': -0.9965880723953249,
 'Calcium': 0.0032110334133148197,
 'Iron': -0.0004691434089303016,
 'Potassium': 0.0027104968500137328,
 'Vitamin D': -0.049999933759455134,
 'Water': 0.0,
 'Weight': 2.6627289056777954,
 'Energy/Calories': -0.06572284098336922}

In [98]:
fixed_heavy_dictionary(triall,new_key_list)

-2.287294713701527
Total Fat
Saturated Fat
Trans Fat
Cholesterol
Sodium
Total Carbs
Fiber
Total Sugar
Protein
Calcium
Iron
Potassium
Vitamin D
Water
Weight


{'Total Fat': 0.0998257279992103,
 'Saturated Fat': -0.6261033887863157,
 'Trans Fat': 0.0,
 'Cholesterol': -0.009868832118809227,
 'Sodium': -0.12654484031528235,
 'Total Carbs': -0.3619944262504582,
 'Fiber': -0.3315937614440919,
 'Total Sugar': -0.07734439848661445,
 'Protein': -1.1542198194265367,
 'Calcium': 0.002497422463798523,
 'Iron': -0.0004953165926843882,
 'Potassium': 0.0010406926643848399,
 'Vitamin D': -0.04999996871490541,
 'Water': 0.0,
 'Weight': 0.34750619530677795}

In [80]:
def error_for_light_fixing_heavy(light_ingredients,optimized_heavy_ingredients):

    desired_dict = fixed_heavy_dictionary(optimized_heavy_ingredients,key_list_API)

    equations_list = equations(tates_ingredient_dict,ingredients[-3:],light_ingredients,desired_dict,key_list_no_weight)
    
    error = 100 * (equations_list[4] ** 2)

    #see if I need this, or how much weight this is given to error
    for i in range(1,3):
        if light_ingredients[-i] - light_ingredients[-i - 1] > 0:
            error = error + ((light_ingredients[-i] - light_ingredients[-i - 1]) ** 2)

    #equations_list = np.array(equations_list, dtype=np.float64)


    #error = tf.reduce_mean(tf.square(equations_list))
    error += sum(equations_list)

    return error

In [82]:
error_for_light_fixing_heavy([.0,.0,.0],trial)

1.8869396080536853


2.2645757789163024

In [83]:
# Map lighter ingredients to their indices
lighter_indices = [6, 7, 8]  # vanilla, baking soda, xanthan gum

# Define the search space for Bayesian Optimization using the indices
search_space = [
    Real(0.0000001, 1, name=f"ingredient_{lighter_indices[0]}"),  # baking_soda
    Real(0.0000001, 1, name=f"ingredient_{lighter_indices[1]}"),  # salt
    Real(0.0000001, 1, name=f"ingredient_{lighter_indices[2]}")   # vanilla
]

def bayesiann(optimized_heavy_ingredients):
    print("Optimized heavy ingredients passed to bayesiann:")
    print(optimized_heavy_ingredients)

    error_function_fixed = partial(error_for_light_fixing_heavy, optimized_heavy_ingredients=optimized_heavy_ingredients)

    result = gp_minimize(
    func=error_function_fixed,  # Pass the function to minimize
    dimensions=search_space,
    n_calls=100,
    n_random_starts=10,
    random_state=410  # Uncomment for reproducibility
)
    print("Trial History (Guessed Inputs and Corresponding Losses):")
    for i, (inputs, loss) in enumerate(zip(result.x_iters, result.func_vals)):
        print(f"Trial {i+1}: Inputs = {inputs}, Loss = {loss}")

    optimized_lighter_ingredients = result.x
    print("Debug: Optimized lighter ingredients from Bayesian Optimization:")
    print(result.x)

    print("Debug: Minimum error achieved:")
    print(result.fun)
    return optimized_lighter_ingredients

In [84]:
bayesiann(trial) #error divided by 10

Optimized heavy ingredients passed to bayesiann:
[6.687910079956055, 6.379572868347168, 6.321332931518555, 5.336599349975586, 4.272467613220215, 1.6648460626602173]
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853


/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1e-07] before, using random point [0.2166229906357163, 0.15783411441752448, 0.751873798807828]
  warnings.warn(
/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1e-07] before, using random point [0.5743838814292236, 0.14819688556044272, 0.48903172923317806]
  warnings.warn(


1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853


/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1e-07] before, using random point [0.6159393223971137, 0.5311106834427753, 0.8091473100908627]
  warnings.warn(


1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853
1.8869396080536853


...

/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1e-07] before, using random point [0.6342417611893675, 0.12994526527580735, 0.8890156467215342]
  warnings.warn(


1.8869396080536853
1.8869396080536853
1.8869396080536853


/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1e-07] before, using random point [0.9050339030786307, 0.8731462280539649, 0.15200171699304788]
  warnings.warn(


1.8869396080536853
Trial History (Guessed Inputs and Corresponding Losses):
Trial 1: Inputs = [0.5574396817667324, 0.5279668591641179, 0.8561068083898229], Loss = 26.585295079135363
Trial 2: Inputs = [0.7539295569749173, 0.03886934215416717, 0.013633090209335936], Loss = 12.823358580136874
Trial 3: Inputs = [0.4633875429059946, 0.5998915348310422, 0.08766771214737554], Loss = 24.53286662597847
Trial 4: Inputs = [0.5338848704456247, 0.22192046354122805, 0.7596315432253532], Loss = 15.96884426101283
Trial 5: Inputs = [0.6321626748377455, 0.4943105625354395, 0.09026948893971057], Loss = 25.041114099783375
Trial 6: Inputs = [0.8445672333479876, 0.23231749941448326, 0.01668420804724873], Loss = 20.76851870149849
Trial 7: Inputs = [0.7059847559792597, 0.09417683337930893, 0.47352110779145984], Loss = 14.848450321852377
Trial 8: Inputs = [0.8488952284938576, 0.883370282414654, 0.604652899095959], Loss = 51.51781325366062
Trial 9: Inputs = [0.01724983185009452, 0.607989941634221, 0.03580685495

[1e-07, 1e-07, 1e-07]

In [253]:
bayesiann(trial) #error divided by 10

Optimized heavy ingredients passed to bayesiann:
[6.687910079956055, 6.379572868347168, 6.321332931518555, 5.336599349975586, 4.272467613220215, 1.6648460626602173]
Trial History (Guessed Inputs and Corresponding Losses):
Trial 1: Inputs = [0.5574396817667324, 0.5279668591641179, 0.8561068083898229], Loss = 0.497070085111368
Trial 2: Inputs = [0.7539295569749173, 0.03886934215416717, 0.013633090209335936], Loss = 0.13179294040147363
Trial 3: Inputs = [0.4633875429059946, 0.5998915348310422, 0.08766771214737554], Loss = 0.13723120893896004
Trial 4: Inputs = [0.5338848704456247, 0.22192046354122805, 0.7596315432253532], Loss = 0.4149854784991331
Trial 5: Inputs = [0.6321626748377455, 0.4943105625354395, 0.09026948893971057], Loss = 0.13759215229460967
Trial 6: Inputs = [0.8445672333479876, 0.23231749941448326, 0.01668420804724873], Loss = 0.13389118439484532
Trial 7: Inputs = [0.7059847559792597, 0.09417683337930893, 0.47352110779145984], Loss = 0.2377476959141136
Trial 8: Inputs = [0.84

[0.42929046134930265, 0.011391816692294913, 0.02286443167391452]

In [189]:
bayesiann(trial) #error divided by nothing

Optimized heavy ingredients passed to bayesiann:
[6.569365978240967, 5.498148441314697, 5.273582935333252, 5.364184379577637, 4.248894691467285, 3.4599080085754395]


/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1.0] before, using random point [0.542077266451464, 0.6176488728090731, 0.8124382379430596]
  warnings.warn(


Trial History (Guessed Inputs and Corresponding Losses):
Trial 1: Inputs = [0.5574396817667324, 0.5279668591641179, 0.8561068083898229], Loss = 3.0036423428909083
Trial 2: Inputs = [0.7539295569749173, 0.03886934215416717, 0.013633090209335936], Loss = 5.434281513603801
Trial 3: Inputs = [0.4633875429059946, 0.5998915348310422, 0.08766771214737554], Loss = 5.19396377489779
Trial 4: Inputs = [0.5338848704456247, 0.22192046354122805, 0.7596315432253532], Loss = 3.2417800149891707
Trial 5: Inputs = [0.6321626748377455, 0.4943105625354395, 0.09026948893971057], Loss = 5.185687963400658
Trial 6: Inputs = [0.8445672333479876, 0.23231749941448326, 0.01668420804724873], Loss = 5.426208716817941
Trial 7: Inputs = [0.7059847559792597, 0.09417683337930893, 0.47352110779145984], Loss = 4.0140365712256285
Trial 8: Inputs = [0.8488952284938576, 0.883370282414654, 0.604652899095959], Loss = 3.66308818358855
Trial 9: Inputs = [0.01724983185009452, 0.607989941634221, 0.03580685495567434], Loss = 5.3610

[0.46113187899535485, 1e-07, 1.0]

In [176]:
bayesiann(trial) #error divided by nothing

Optimized heavy ingredients passed to bayesiann:
[6.569365978240967, 5.498148441314697, 5.273582935333252, 5.364184379577637, 4.248894691467285, 3.4599080085754395]
Trial History (Guessed Inputs and Corresponding Losses):
Trial 1: Inputs = [0.5574396817667324, 0.5279668591641179, 0.8561068083898229], Loss = 2.999810082515677
Trial 2: Inputs = [0.7539295569749173, 0.03886934215416717, 0.013633090209335936], Loss = 5.433621804004859
Trial 3: Inputs = [0.4633875429059946, 0.5998915348310422, 0.08766771214737554], Loss = 5.190061740747148
Trial 4: Inputs = [0.5338848704456247, 0.22192046354122805, 0.7596315432253532], Loss = 3.2409661482496923
Trial 5: Inputs = [0.6321626748377455, 0.4943105625354395, 0.09026948893971057], Loss = 5.181606375590632
Trial 6: Inputs = [0.8445672333479876, 0.23231749941448326, 0.01668420804724873], Loss = 5.423471245717233
Trial 7: Inputs = [0.7059847559792597, 0.09417683337930893, 0.47352110779145984], Loss = 4.013258166164081
Trial 8: Inputs = [0.84889522849

[0.8308854257591127, 1e-07, 1.0]

In [220]:
bayesiann(trial) #changed the error function

Optimized heavy ingredients passed to bayesiann:
[6.569365978240967, 5.498148441314697, 5.273582935333252, 5.364184379577637, 4.248894691467285, 3.4599080085754395]


/Users/shivanishah/anaconda3/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-07, 1e-07, 1.0] before, using random point [0.542077266451464, 0.6176488728090731, 0.8124382379430596]
  warnings.warn(


Trial History (Guessed Inputs and Corresponding Losses):
Trial 1: Inputs = [0.5574396817667324, 0.5279668591641179, 0.8561068083898229], Loss = 3.0036423428909083
Trial 2: Inputs = [0.7539295569749173, 0.03886934215416717, 0.013633090209335936], Loss = 5.434281513603801
Trial 3: Inputs = [0.4633875429059946, 0.5998915348310422, 0.08766771214737554], Loss = 5.19396377489779
Trial 4: Inputs = [0.5338848704456247, 0.22192046354122805, 0.7596315432253532], Loss = 3.2417800149891707
Trial 5: Inputs = [0.6321626748377455, 0.4943105625354395, 0.09026948893971057], Loss = 5.185687963400658
Trial 6: Inputs = [0.8445672333479876, 0.23231749941448326, 0.01668420804724873], Loss = 5.426208716817941
Trial 7: Inputs = [0.7059847559792597, 0.09417683337930893, 0.47352110779145984], Loss = 4.0140365712256285
Trial 8: Inputs = [0.8488952284938576, 0.883370282414654, 0.604652899095959], Loss = 3.66308818358855
Trial 9: Inputs = [0.01724983185009452, 0.607989941634221, 0.03580685495567434], Loss = 5.3610

[0.46113187899535485, 1e-07, 1.0]

In [154]:
def reversy(dictionary_list_for_heavy,target_dict,key_list):

    loss, optimized_heavy = running_tf(dictionary_list_for_heavy,target_dict,key_list)
    noisy_optimized = add_noise(optimized_heavy)
    lighter = bayesiann(noisy_optimized)

    return optimized_heavy, lighter, loss
    